<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:**
Свойства матриц, необходимые для обоснования:
$$
1) P*P^{-1}=E
$$


$$
2) (AB)^{-1} = B^{-1}A^{-1}
$$


$$
3) (AB)^T = B^TA^T
$$


$$
4) AE = A
$$


Подставим в формулы $w = (X^T X)^{-1} X^T y$ и $a = Xw$ вместо $X$ произведение $X$ и $P$, получим:

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
a = XPw
$$
Объединим формулы:

$$
a = XP((XP)^T XP)^{-1} (XP)^T y
$$

Используя 3-е свойство, раскроем $((XP)^T$:

$$
a = XP((XP)^T XP)^{-1} (XP)^T y =  XP(P^TX^TXP)^{-1}P^TX^T y
$$

Используя 2-е свойство, раскроем $(P^TX^TXP)^{-1}$:

$$
a = XP(P^TX^TXP)^{-1}P^TX^T y = XPP^-{1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y
$$

Произведение $P$ и $P^{-1}$ $=$ $E$ и произведение $P^T$ и $(P^T)^{-1}$ $=$ $E$ (1-e свойство). Получаем:

$$
a = XPP^-{1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y =  XE(X^TX)^{-1}EX^T
$$

Благодаря 4-му свойству $XE = X$. В итоге:

$$
a =  XE(X^TX)^{-1}EX^T = X(X^TX)^{-1}X^T y = Xw
$$

## Алгоритм преобразования

**Алгоритм**

Для данных клиентов страховой компании матрицу признаков умножим на обратимую матрицу, которую создадим случайным образом. Данные клиентов будут преобразованы. Для этого:
1. Выделим из исходных данных признаки и целевой признак.
2. Создадим обратимую матрицу(*A*) размером  $n∗n$ , равным ширине матрице исходных признаков (количесвтво строк новой матрицы должно быть равно количеству столбцов исходной, чтобы выполнылось матричное умножение).
3. Умножим матрицу исходных признаков на созданную обратимую матрицу (*A*).

**Обоснование**

Как было доказано выше, произведение признаков на обратимую матрицу не изменит качество линейной регрессии.

## Проверка алгоритма

Выделим из исходных данных признаки и целевой признак.

In [6]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

Разделим выборку на обучающую и тестовую.

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Создадим обратимую матрицу.

In [8]:
n = features.shape[1]
A = np.random.randint(100, size=(n, n))

Проверим на обратимость.

In [9]:
np.linalg.inv(A)

array([[-9.97065481e-03,  7.82587123e-03,  3.66889443e-05,
         8.72453327e-03],
       [ 1.07331444e-03, -2.04656467e-02,  1.74880350e-03,
         1.05353715e-02],
       [ 7.63629359e-05,  1.41689338e-02,  1.62045189e-02,
        -1.19931600e-02],
       [ 1.09477422e-02,  1.00016450e-02, -9.46813568e-03,
        -2.97674339e-03]])

Обратная матрица для нашей созданной матрицу (*A*) существует, поэтому матрица (*A*) обратима.

Создим преобразованную матрицу признаков.

In [10]:
features_new = features@A

Обучим модель линейной регресси на исходных данных.

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
predict = model.predict(features_test)
r2 = r2_score(target_test, predict)

Обучим модель линейной регрессии на преобразованных данных.

In [12]:
features_train, features_test, target_train, target_test = train_test_split(
    features_new, target, test_size=0.25, random_state=12345)

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predict = model.predict(features_test)
r2_new = r2_score(target_test, predict)

Сравним качество двух моделей.

In [14]:
print(r2)
print(r2_new)

0.435227571270266
0.43522757127025036


Как видим, качество модели на преобразованных данных не изменилось. 

Чтобы декодировать матрицу признаков необходимо преобразованную матрицу умножить на обратную матрицу, на которую умножали исходные признаки, ввиду свойства: $A*A^{-1}=E$.

In [18]:
abs(round(features_test@np.linalg.inv(A)))

,0,1,2,3
3183,0.0,33.0,39000.0,4.0
1071,0.0,50.0,43100.0,2.0
2640,1.0,39.0,42100.0,0.0
2282,0.0,20.0,34800.0,0.0
1595,0.0,41.0,40000.0,4.0
...,...,...,...,...
982,1.0,51.0,29000.0,2.0
3820,1.0,33.0,57900.0,3.0
3595,1.0,35.0,42300.0,0.0
3513,1.0,36.0,41300.0,0.0


## Вывод

Таким образом, в ходе работы установленно, что качество линейной регрессии не измениться, если преобразовать исходные данные путем их умножения на обратимую матрицу, что доказано теоретически и подтверждено практически.

Данные можно зашифровать путем их умножения на обратимую матрицу.